In [ ]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving dep2.csv to dep2.csv


## Data Pre-Processing

In [ ]:
import io
dep = pd.read_csv(io.BytesIO(data_to_load['dep2.csv']))

X = dep.drop("depressed", axis=1)
y = dep["depressed"]
print(X.shape, y.shape)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

(1147, 34) (1147,)


# Create a Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='elu', input_dim=34))
model.add(Dense(units=100, activation='elu'))

model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
27/27 - 0s - loss: 0.4925 - accuracy: 0.8174
Epoch 2/100
27/27 - 0s - loss: 0.4569 - accuracy: 0.8326
Epoch 3/100
27/27 - 0s - loss: 0.4459 - accuracy: 0.8326
Epoch 4/100
27/27 - 0s - loss: 0.4474 - accuracy: 0.8314
Epoch 5/100
27/27 - 0s - loss: 0.4388 - accuracy: 0.8314
Epoch 6/100
27/27 - 0s - loss: 0.4360 - accuracy: 0.8326
Epoch 7/100
27/27 - 0s - loss: 0.4350 - accuracy: 0.8326
Epoch 8/100
27/27 - 0s - loss: 0.4349 - accuracy: 0.8291
Epoch 9/100
27/27 - 0s - loss: 0.4314 - accuracy: 0.8337
Epoch 10/100
27/27 - 0s - loss: 0.4325 - accuracy: 0.8326
Epoch 11/100
27/27 - 0s - loss: 0.4315 - accuracy: 0.8326
Epoch 12/100
27/27 - 0s - loss: 0.4307 - accuracy: 0.8372
Epoch 13/100
27/27 - 0s - loss: 0.4313 - accuracy: 0.8326
Epoch 14/100
27/27 - 0s - loss: 0.4301 - accuracy: 0.8384
Epoch 15/100
27/27 - 0s - loss: 0.4330 - accuracy: 0.8326
Epoch 16/100
27/27 - 0s - loss: 0.4267 - accuracy: 0.8349
Epoch 17/100
27/27 - 0s - loss: 0.4267 - accuracy: 0.8337
Epoch 18/100
27/27 - 0s

## Quantify our Trained Model

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 0.5274 - accuracy: 0.8223
Normal Neural Network - Loss: 0.5274163484573364, Accuracy: 0.8222996592521667


# Saving a Trained Model
We can save our trained models using the HDF5 binary format with the extension `.h5`

In [ ]:
# Save the model
model.save("dep_model_trained.h5")

# Loading a Model

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
dep_model = load_model("dep_model_trained.h5")

## Evaluating the loaded model

In [ ]:
#model_loss, model_accuracy = dep_model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
#print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
model_loss, model_accuracy = dep_model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 0.5274 - accuracy: 0.8223
Loss: 0.5274163484573364, Accuracy: 0.8222996592521667


In [ ]:
#predictions = dep_model.predict_classes(X_test_scaled)

In [ ]:
predictions = dep_model.predict_classes(X_test_scaled)

In [ ]:
predictionvsactual = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

In [ ]:
pv = predictionvsactual.reset_index(drop=True)
pv

,Prediction,Actual
0,0,0.0
1,0,1.0
2,0,0.0
3,0,1.0
4,0,0.0
...,...,...
282,0,1.0
283,0,0.0
284,0,1.0
285,0,0.0


In [ ]:
predictionvsactual.to_csv('pva.csv') 

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
  
actual = y_test
predicted = predictions
results = confusion_matrix(predicted,actual)

In [ ]:
results

array([[233,  47],
       [  4,   3]])

In [ ]:
sensitivity1 = results[1,1]/(results[0,1]+results[1,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = results[0,0]/(results[0,0]+results[1,0])
print('Specificity : ', specificity1)

Sensitivity :  0.06
Specificity :  0.9831223628691983


In [ ]:
data = np.array(X_test)

In [ ]:
predictionvsactual2 = pd.DataFrame(data)

In [ ]:
predictionvsactual2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,1.0,63.0,1.0,0.0,4.0,8.0,0.0,154.258510,174.567540,0.000000,0.000000,2.50,72.339905,0.000000,0.000000,18.417675,0.000000,0.000000,26.425360,37.075581,35.233814,7.687377,15.896144,0.0,0.000000,2.000000,0.0,0.0,4.500000,0.0,0.000000,0.000000,256.878300,15.896145
1,1.0,28.0,1.0,4.0,6.0,9.0,4.0,256.999020,46.444572,32.030739,0.000000,0.00,230.627040,1.174514,0.632551,0.000000,0.000000,4.003842,0.266923,22.101212,0.000000,0.000000,3.603458,3.0,0.000000,1.000000,0.0,1.0,0.500000,0.0,48.046108,0.750000,217.871280,7.607301
2,1.0,20.0,1.0,3.0,5.0,9.0,3.0,82.972580,105.701440,24.023054,3.203074,0.00,40.798771,0.000000,0.000000,0.000000,0.000000,0.000000,0.533846,41.639961,57.655331,0.800768,1.757242,0.0,0.000000,10.000000,1.0,1.0,6.000000,0.0,0.000000,0.000000,143.548570,4.960316
3,1.0,24.0,0.0,1.0,4.0,10.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,2.409585,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.549550,3.068881,0.0,0.0,1.756061,0.0,0.000000,0.000000,0.000000,0.000000
4,1.0,26.0,1.0,4.0,6.0,10.0,4.0,194.444520,215.887190,40.038425,12.812296,0.00,133.212420,0.000000,0.000000,0.000000,0.000000,1.561498,0.093423,59.577175,230.621320,115.310660,116.645280,0.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,18.737984,0.666667,253.536040,131.019070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,1.0,67.0,0.0,0.0,1.0,1.0,0.0,37.013428,32.831509,0.000000,0.000000,1.75,25.749285,0.000000,0.000000,0.000000,0.000000,0.000000,2.535767,8.728376,0.000000,0.000000,4.804611,3.0,0.000000,1.000000,0.0,1.0,3.000000,0.0,0.000000,0.000000,58.287693,4.804611
283,1.0,27.0,1.0,3.0,5.0,10.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,1.259431,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.353591,3.068881,0.0,0.0,1.612739,0.0,0.000000,0.000000,0.000000,0.000000
284,1.0,27.0,1.0,5.0,7.0,9.0,5.0,190.823140,174.567540,16.015369,0.000000,0.25,161.194700,0.000000,0.000000,0.000000,0.000000,3.843689,0.320307,25.464437,14.413833,16.015369,16.015369,0.0,0.000000,3.000000,0.0,0.0,3.857143,0.0,46.124264,0.750000,190.835850,19.859058
285,1.0,35.0,1.0,4.0,6.0,10.0,4.0,159.829210,100.416370,19.218445,0.000000,0.00,129.165100,0.000000,0.000000,0.000000,0.000000,0.448430,2.428998,27.786667,67.264557,18.364290,22.901979,0.0,0.000000,7.000000,0.0,1.0,2.000000,0.0,5.381165,0.750000,157.755860,23.350409


In [ ]:
frames = [pv, predictionvsactual2]
result = pd.concat(frames, axis=1)

In [ ]:
result.head(50)

,Prediction,Actual,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,0,0.0,1.0,63.0,1.0,0.0,4.0,8.0,0.0,154.258510,174.567540,0.000000,0.000000,2.5,72.339905,0.000000,0.000000,18.417675,0.000000,0.000000,26.425360,37.075581,35.233814,7.687377,15.896144,0.0,0.000000,2.000000,0.0,0.0,4.500000,0.0,0.000000,0.000000,256.878300,15.896145
1,0,1.0,1.0,28.0,1.0,4.0,6.0,9.0,4.0,256.999020,46.444572,32.030739,0.000000,0.0,230.627040,1.174514,0.632551,0.000000,0.000000,4.003842,0.266923,22.101212,0.000000,0.000000,3.603458,3.0,0.000000,1.000000,0.0,1.0,0.500000,0.0,48.046108,0.750000,217.871280,7.607301
2,0,0.0,1.0,20.0,1.0,3.0,5.0,9.0,3.0,82.972580,105.701440,24.023054,3.203074,0.0,40.798771,0.000000,0.000000,0.000000,0.000000,0.000000,0.533846,41.639961,57.655331,0.800768,1.757242,0.0,0.000000,10.000000,1.0,1.0,6.000000,0.0,0.000000,0.000000,143.548570,4.960316
3,0,1.0,1.0,24.0,0.0,1.0,4.0,10.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,2.409585,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.549550,3.068881,0.0,0.0,1.756061,0.0,0.000000,0.000000,0.000000,0.000000
4,0,0.0,1.0,26.0,1.0,4.0,6.0,10.0,4.0,194.444520,215.887190,40.038425,12.812296,0.0,133.212420,0.000000,0.000000,0.000000,0.000000,1.561498,0.093423,59.577175,230.621320,115.310660,116.645280,0.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,18.737984,0.666667,253.536040,131.019070
5,0,0.0,1.0,21.0,1.0,4.0,6.0,10.0,4.0,97.438652,221.812880,20.819981,0.000000,0.0,65.036133,6.978126,0.000000,0.000000,0.000000,1.868460,0.734038,22.821901,0.000000,0.000000,5.772206,0.0,0.000000,2.000000,0.0,0.0,0.000000,0.0,22.421518,1.000000,570.829100,7.640666
6,0,0.0,1.0,27.0,1.0,1.0,3.0,12.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,2.409585,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.549550,3.068881,0.0,0.0,2.495495,0.0,0.000000,0.000000,0.000000,0.000000
7,0,0.0,1.0,19.0,1.0,2.0,4.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.792240,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.224138,3.068881,0.0,0.0,1.756061,0.0,0.000000,0.000000,0.000000,0.000000
8,0,0.0,1.0,44.0,0.0,4.0,6.0,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.217299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.504098,3.068881,0.0,0.0,1.438596,0.0,0.000000,0.000000,0.000000,0.000000
9,0,0.0,1.0,24.0,1.0,4.0,6.0,9.0,4.0,118.847770,198.270280,24.023054,4.804611,2.0,91.901909,0.000000,0.000000,0.000000,0.000000,2.135383,2.068652,22.741825,0.000000,0.000000,0.000000,0.0,0.000000,2.000000,1.0,0.0,0.500000,0.0,25.624592,1.000000,273.475160,6.939994


In [ ]:
result.to_csv('pva2.csv')